In [ ]:
import json
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
results = [json.load(open(f)) for f in glob.glob('2020*/coco_val2017_rcnn_3x_q_[1-9]*.json')]

In [ ]:
res = pd.Series({r['quality']: r['elapsed'] for r in results}).to_frame(name='elapsed').sort_index()
res['batch'] = res.index % 4
for b in range(4):
    res[f'e{b}'] = np.nan
    res.loc[res.batch == b, f'e{b}'] = res.elapsed
    res[f'e{b}'] = res[f'e{b}'].interpolate()
del res['batch']
del res['elapsed'] # noisy
res.plot();

In [ ]:
df = pd.Series({r['quality']: r['bbox']['AP50'] for r in results}, name='AP50').sort_index().to_frame()
df['AP'] = pd.Series({r['quality']: r['bbox']['AP'] for r in results}).sort_index()
df.index.name = 'Q'
df.tail()

In [ ]:
plot = df.plot()
plot.get_figure().canvas.get_supported_filetypes()

In [ ]:
fig = plot.get_figure()
fig.set_size_inches(3, 2)
fig.savefig('ap_vs_ap50.pdf')
fig.savefig('ap_vs_ap50.pgf')

In [ ]:
from ipywidgets import interact
ddf = df.diff()
@interact(x=(-0.13, 0, 0.01))
def bad_vals(x):
    return ddf[(ddf.AP < x) | (ddf.AP50 < x)].reset_index()

In [ ]:
#ddf.columns #= ['d(AP50)', 'd(AP)']
ax = ddf.plot()
ax.axhline(0, ls='dotted')

In [ ]:
results_cardio = [json.load(open(f)) for f in glob.glob('../cardio_results/*.json')]

In [ ]:
results_cardio[0]

In [ ]:
from itertools import groupby
gb = lambda it, key: groupby(sorted(it, key=key), key)

for key, group in gb(results_cardio, lambda x: x['model_config']):
    print(key)

In [ ]:
sum(r['elapsed'] for r in results_cardio)

In [ ]:
def flatten(value):
    result = value.copy()
    result.update(result['bbox'])
    del result['bbox']
    return result

c_df = pd.DataFrame([flatten(r) for r in results_cardio]).sort_values(['model_config', 'quality'])
c_df.head()

In [ ]:
for mc in c_df.model_config.unique():
    ax = c_df[c_df.model_config == mc][['quality', 'AP', 'AP50', 'AP75']].plot(x='quality')
    ax.set_title(mc)

In [ ]:
ax = c_df.pivot(index='quality', columns='model_config', values='AP').plot()
ax.set_title('AP between models')

In [ ]:
ax = c_df.pivot(index='quality', columns='model_config', values='AP50').plot()
ax.set_title('AP50 between models')

In [ ]:
for mc in c_df.model_config.unique():
    ax = c_df[c_df.model_config == mc][['quality', 'APs', 'APm', 'APl']].plot(x='quality')
    ax.set_title(mc)